In [1]:
import datajoint as dj
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd
import ray

sys.path.append('./lib')
from utilities import *
sys.path.append('../lib')
from utils import run

In [2]:
def setup_download_from_s3( rel_fp, recursive=True ):
    s3_fp = 's3://mousebrainatlas-data/'+rel_fp
    local_fp = os.environ['ROOT_DIR']+rel_fp

    if os.path.exists(local_fp):
        print('ALREADY DOWNLOADED FILE')
        return 
    
    if recursive:
        ! aws s3 cp --recursive $s3_fp $local_fp
    else:
        ! aws s3 cp $s3_fp $local_fp

In [5]:
setup_download_from_s3('CSHL_hsv/MD589/VLL')

download: s3://mousebrainatlas-data/CSHL_hsv/MD589/VLL/VLL_131.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/VLL/VLL_131.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/VLL/VLL_132.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/VLL/VLL_132.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/VLL/VLL_134.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/VLL/VLL_134.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/VLL/VLL_133.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/VLL/VLL_133.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/VLL/VLL_136.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/VLL/VLL_136.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/VLL/VLL_135.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/VLL/VLL_135.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/VLL/VLL_138.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/V

download: s3://mousebrainatlas-data/CSHL_hsv/MD589/VLL/VLL_317.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/VLL/VLL_317.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/VLL/VLL_318.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/VLL/VLL_318.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/VLL/VLL_319.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/VLL/VLL_319.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/VLL/VLL_321.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/VLL/VLL_321.tif
download failed: s3://mousebrainatlas-data/CSHL_hsv/MD589/VLL/VLL_304.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/VLL/VLL_304.tif Could not connect to the endpoint URL: "https://mousebrainatlas-data.s3.us-west-1.amazonaws.com/CSHL_hsv/MD589/VLL/VLL_304.tif"
download failed: s3://mousebrainatlas-data/CSHL_hsv/MD589/VLL/VLL_168.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/VLL/

In [2]:
def setup_upload_from_s3(rel_fp, recursive=True):
    s3_fp = 's3://mousebrainatlas-data/' + rel_fp
    local_fp = os.environ['ROOT_DIR'] + rel_fp

    if recursive:
        run('aws s3 cp --recursive {0} {1}'.format(local_fp, s3_fp))
    else:
        run('aws s3 cp {0} {1}'.format(local_fp, s3_fp))

In [3]:
setup_upload_from_s3('CSHL_patches_features/')

run cmd= aws s3 cp --recursive /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_features/ s3://mousebrainatlas-data/CSHL_patches_features/


In [2]:
#credFiles= '/data/Github/VaultBrain/credFiles_aws.yaml'
credFiles= '/Users/kuiqian/Github/VaultBrain/credFiles.yaml'
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
dj.conn()

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


Connecting kui@ucsd-demo-db.datajoint.io:3306


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [3]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [4]:
stack = 'MD589'
#yaml_file = 'shape_params-aws.yaml'
yaml_file = 'shape_params.yaml'
img_file = '/CSHL_hsv/'+stack+'/'
scripts_dir = os.environ['REPO_DIR']

In [28]:
Structure.drop()

`kui_diffusionmap`.`structure` (28 tuples)
`kui_diffusionmap`.`__process` (6 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [29]:
@schema
class Structure(dj.Manual):
    definition = """
    id  : int   # sequence
    --------
    structure       : char(10)    # name of mouse brain's structure
    """

In [30]:
paired_structures = ['5N', '6N', '7nn', '7N', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', \
                     'SNC', 'SNR', '3N', '4N', 'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC'] #
all_structures = paired_structures + singular_structures
n=0
for struc in all_structures:
    print("\nAdding "+struc+' to the database')
    Structure.insert1(dict(id=n,
                           structure=struc,)
                      ,skip_duplicates=False)
    n+=1


Adding 5N to the database

Adding 6N to the database

Adding 7nn to the database

Adding 7N to the database

Adding Amb to the database

Adding LC to the database

Adding LRt to the database

Adding Pn to the database

Adding Tz to the database

Adding VLL to the database

Adding RMC to the database

Adding SNC to the database

Adding SNR to the database

Adding 3N to the database

Adding 4N to the database

Adding Sp5I to the database

Adding Sp5O to the database

Adding Sp5C to the database

Adding PBG to the database

Adding 10N to the database

Adding VCA to the database

Adding VCP to the database

Adding DC to the database

Adding AP to the database

Adding 12N to the database

Adding RtTg to the database

Adding SC to the database

Adding IC to the database


In [31]:
structureTable = Structure.fetch(as_dict=True)
strucDF = pd.DataFrame(structureTable)
strucDF

,id,structure
0,0,5N
1,1,6N
2,2,7nn
3,3,7N
4,4,Amb
5,5,LC
6,6,LRt
7,7,Pn
8,8,Tz
9,9,VLL


In [15]:
Process.drop()

`kui_diffusionmap`.`__process` (4 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [32]:
@schema
class Process(dj.Computed):
    definition="""
    -> Structure
    -----
    image_number : int   #number of images
    """

    bucket = "mousebrainatlas-data"
    client = get_s3_client(credFiles)
    def make(self, key):
        struc = (Structure & key).fetch1('structure')
        print('populating for ', struc, end='\n')
        key_item = 'image_number'
        s3_fp = img_file+struc+'/'
        upload_fp = 's3://'+self.bucket+s3_fp
        pkl_out_file = os.environ['ROOT_DIR']+s3_fp
        try:
            objects = os.listdir(pkl_out_file)
            key[key_item] = len(objects)
        except:
            ! python3 $scripts_dir/HSV.py $stack $struc $yaml_file
            ! aws s3 cp --recursive $pkl_out_file $upload_fp
            objects = os.listdir(pkl_out_file)
            key[key_item] = len(objects)
        self.insert1(key)


./lib/utilities.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


In [63]:
Process.populate(reserve_jobs=True) 

populating for  IC


In [64]:
Process.fetch(as_dict=True)

[OrderedDict([('id', 0), ('image_number', 44)]),
 OrderedDict([('id', 1), ('image_number', 5)]),
 OrderedDict([('id', 2), ('image_number', 125)]),
 OrderedDict([('id', 3), ('image_number', 89)]),
 OrderedDict([('id', 4), ('image_number', 13)]),
 OrderedDict([('id', 5), ('image_number', 24)]),
 OrderedDict([('id', 6), ('image_number', 76)]),
 OrderedDict([('id', 7), ('image_number', 100)]),
 OrderedDict([('id', 8), ('image_number', 50)]),
 OrderedDict([('id', 9), ('image_number', 69)]),
 OrderedDict([('id', 10), ('image_number', 57)]),
 OrderedDict([('id', 11), ('image_number', 80)]),
 OrderedDict([('id', 12), ('image_number', 126)]),
 OrderedDict([('id', 13), ('image_number', 24)]),
 OrderedDict([('id', 14), ('image_number', 11)]),
 OrderedDict([('id', 15), ('image_number', 73)]),
 OrderedDict([('id', 16), ('image_number', 67)]),
 OrderedDict([('id', 17), ('image_number', 141)]),
 OrderedDict([('id', 18), ('image_number', 24)]),
 OrderedDict([('id', 19), ('image_number', 45)]),
 Ordere

In [61]:
(schema.jobs & 'status="reserved"' ).delete()

In [62]:
schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
